In [1]:
from utils.misc import *
from train.train3 import Train,Test
from model.Creat_model import creat_model
from utils.preprocess import *
from utils.utils import *

2025-07-09 10:39:42.907768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-09 10:39:43.714382: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 10:39:48.074597: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:
2025-07-09 10:39:48.083672: W tensorflow/compiler/xla/stream_executor/p

In [2]:
from torch.backends import cudnn
def fix_seed(seed):
    #seed = 2023
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'   

In [3]:
import networkx
import torch
import scipy.sparse as sp
import numpy as np
import os
import random
from munkres import Munkres
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
import argparse
import yaml
from torch_geometric.utils.convert import to_networkx
import pynvml

def build_args():
    
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--dataset', type=str, default="acm")
    parser.add_argument('--seed', type=int, default=20)
    parser.add_argument('--cuda', type=bool, default=True)
    parser.add_argument('--n_input', type=int, default=None)
    parser.add_argument('--n_z', type=int, default=None)
    parser.add_argument('--freedom_degree', type=float, default=1.0)
    parser.add_argument('--epoch', type=int, default=None)
    parser.add_argument('--shuffle', type=bool, default=True)
    parser.add_argument('--sigma', type=float, default=None)
    parser.add_argument('--loss_n', type=float, default=None)
    parser.add_argument('--loss_w', type=float, default=None)
    parser.add_argument('--loss_s', type=float, default=None)
    parser.add_argument('--loss_a', type=float, default=None)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--acc', type=float, default=-1)
    parser.add_argument('--f1', type=float, default=-1)
    args = parser.parse_args([])
    return args

In [4]:
def pca(adata, use_reps=None, n_comps=10):
    
    """Dimension reduction with PCA algorithm"""
    
    from sklearn.decomposition import PCA
    from scipy.sparse.csc import csc_matrix
    from scipy.sparse.csr import csr_matrix
    pca = PCA(n_components=n_comps)
    if use_reps is not None:
       feat_pca = pca.fit_transform(adata.obsm[use_reps])
    else: 
       if isinstance(adata.X, csc_matrix) or isinstance(adata.X, csr_matrix):
          feat_pca = pca.fit_transform(adata.X.toarray()) 
       else:   
          feat_pca = pca.fit_transform(adata.X)
    
    return feat_pca

def clr_normalize_each_cell(adata, inplace=True):
    
    """Normalize count vector for each cell, i.e. for each row of .X"""

    import numpy as np
    import scipy

    def seurat_clr(x):
        # TODO: support sparseness
        s = np.sum(np.log1p(x[x > 0]))
        exp = np.exp(s / len(x))
        return np.log1p(x / exp)

    if not inplace:
        adata = adata.copy()
    
    # apply to dense or sparse matrix, along axis. returns dense matrix
    adata.X = np.apply_along_axis(
        seurat_clr, 1, (adata.X.A if scipy.sparse.issparse(adata.X) else np.array(adata.X))
    )
    return adata     

In [5]:
args = build_args()
args = load_configs(args, "config/configs.yml")
# set_random_seed(2024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load configs


In [6]:
# 淋巴结
import scanpy as sc
adata_omics1 = sc.read_h5ad( '/home/Human_lymph_node/slice3/s3_adata_rna.h5ad')
adata_omics2 = sc.read_h5ad('/home/Human_lymph_node/slice3/s3_adata_adt.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()
adata_omics1 = adata_omics1[adata_omics1.obs['final_annot'].notna()]  
adata_omics2 = adata_omics2[adata_omics2.obs['final_annot'].notna()] 

In [7]:

# RNA
sc.pp.filter_genes(adata_omics1, min_cells=10)
sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1)

adata_omics1 =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1, n_comps=adata_omics2.n_vars-1)
# Protein
adata_omics2 = clr_normalize_each_cell(adata_omics2)
sc.pp.scale(adata_omics2)
adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=adata_omics2.n_vars-1)

/home/hfzhang/software/anaconda3/envs/GraphST/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/tmp/ipykernel_34275/529664883.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix
/tmp/ipykernel_34275/529664883.py:7: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix
/home/hfzhang/software/anaconda3/envs/GraphST/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/tmp/ipykernel_34275/529664883.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace i

In [8]:
def create_adj(adata):
    cell_position_omics1 = adata.obsm['spatial']
    adj_omics1 = construct_graph_by_coordinate(cell_position_omics1, n_neighbors=6)
    adata.uns['adj_spatial'] = adj_omics1
    adj_spatial_omics1 = adata.uns['adj_spatial']
    adj_spatial_omics1 = transform_adjacent_matrix(adj_spatial_omics1)
    adj_spatial_omics1 = adj_spatial_omics1.toarray()
    adj_spatial_omics1 = adj_spatial_omics1 + adj_spatial_omics1.T
    adj_spatial_omics1 = np.where(adj_spatial_omics1>1, 1, adj_spatial_omics1)
    adj = preprocess_graph(adj_spatial_omics1)
    return adj

In [9]:
label=adata_omics2.obs['final_annot'].values
classes, label = np.unique(label, return_inverse=True)
classes = classes.tolist()

In [10]:
args.n_input=adata_omics2.n_vars-1
args.n_input1=adata_omics2.n_vars-1
args.n_clusters=len(classes)
device='cuda:1'
args.n_clusters

args.Type='10x'
args.loss_n=0.001
args.epoch=450
args.n_clusters=len(classes)

In [11]:
random_seek=2024
args.random_seed=random_seek
fix_seed(random_seek)

In [12]:
args.loss_n

0.001

In [13]:
args.sigma=0.6

In [14]:
args.loss_n=0
args.loss_s=0.1
args.tool='kmeans'
args.lr=0.0002

In [15]:
args.n_clusters1=len(set(label))
args.n_clusters2=len(set(label))
adj_train=create_adj(adata_omics1)
adj_train = adj_train.to(device)
features_omics1 = torch.FloatTensor(adata_omics1.obsm['feat'].copy()).to(device)
features_omics2 = torch.FloatTensor(adata_omics2.obsm['feat'].copy()).to(device)

model = creat_model('spamgcn', args).to(device)
model=Train(150, model,adata_omics1, features_omics1,features_omics2, adj_train, label, device, args)
nmi, ari, ami, homogeneity, completeness, v_measure=Test(model,adata_omics1,features_omics1,features_omics2,adj_train,label,device,args,'kmeans')


  9 loss: 11.508434295654297
Epoch_  9 , nmi 0.3544 , ari 0.2266 , ami 0.3501 , homogeneity 0.3772 , completeness 0.3342 , v_measure 0.3544
 19 loss: 11.405802726745605
Epoch_ 19 , nmi 0.3716 , ari 0.2427 , ami 0.3675 , homogeneity 0.4007 , completeness 0.3464 , v_measure 0.3716
 29 loss: 11.233819961547852
Epoch_ 29 , nmi 0.3731 , ari 0.2427 , ami 0.3690 , homogeneity 0.3979 , completeness 0.3512 , v_measure 0.3731
 39 loss: 10.921753883361816
Epoch_ 39 , nmi 0.3727 , ari 0.2427 , ami 0.3686 , homogeneity 0.3954 , completeness 0.3525 , v_measure 0.3727
 49 loss: 10.410407066345215
Epoch_ 49 , nmi 0.3975 , ari 0.2693 , ami 0.3935 , homogeneity 0.4154 , completeness 0.3811 , v_measure 0.3975
 59 loss: 9.762218475341797
Epoch_ 59 , nmi 0.3940 , ari 0.2696 , ami 0.3899 , homogeneity 0.4112 , completeness 0.3781 , v_measure 0.3940
 69 loss: 9.163942337036133
Epoch_ 69 , nmi 0.4020 , ari 0.2923 , ami 0.3978 , homogeneity 0.3995 , completeness 0.4047 , v_measure 0.4020
 79 loss: 8.7408170700

In [16]:
nmi 0.4062 , ari 0.3156 , ami 0.4020 , homogeneity 0.3897 , completeness 0.4242 , v_measure 0.4062  0.6 0.0002 150
nmi 0.3916 , ari 0.3017 , ami 0.3871 , homogeneity 0.3664 , completeness 0.4205 , v_measure 0.3916
nmi 0.3967 , ari 0.3069 , ami 0.3924 , homogeneity 0.3739 , completeness 0.4225 , v_measure 0.3967
nmi 0.4117 , ari 0.3125 , ami 0.4075 , homogeneity 0.4025 , completeness 0.4214 , v_measure 0.4117
nmi 0.4043 , ari 0.3192 , ami 0.4001 , homogeneity 0.3919 , completeness 0.4175 , v_measure 0.4043

SyntaxError: invalid syntax (3494551921.py, line 1)

In [ ]:
nmi 0.4045 , ari 0.3184 , ami 0.4003 , homogeneity 0.3927 , completeness 0.4171 , v_measure 0.4045  0.0002 150 0.5
nmi 0.3948 , ari 0.2999 , ami 0.3906 , homogeneity 0.3782 , completeness 0.4131 , v_measure 0.3948
nmi 0.3998 , ari 0.3149 , ami 0.3955 , homogeneity 0.3823 , completeness 0.4190 , v_measure 0.3998
nmi 0.3909 , ari 0.2977 , ami 0.3864 , homogeneity 0.3650 , completeness 0.4208 , v_measure 0.3909
nmi 0.4010 , ari 0.3076 , ami 0.3969 , homogeneity 0.3778 , completeness 0.4272 , v_measure 0.4010

In [ ]:
nmi 0.4224 , ari 0.3282 , ami 0.4183 , homogeneity 0.4119 , completeness 0.4334 , v_measure 0.4224
nmi 0.4202 , ari 0.3226 , ami 0.4161 , homogeneity 0.4091 , completeness 0.4318 , v_measure 0.4202
nmi 0.4320 , ari 0.3401 , ami 0.4280 , homogeneity 0.4223 , completeness 0.4422 , v_measure 0.4320
nmi 0.4066 , ari 0.3170 , ami 0.4024 , homogeneity 0.3898 , completeness 0.4249 , v_measure 0.4066
nmi 0.4254 , ari 0.3344 , ami 0.4214 , homogeneity 0.4136 , completeness 0.4379 , v_measure 0.4254

SyntaxError: invalid syntax (1606440570.py, line 1)